# Introduction
Welcome to the SBOL developers tutorial. This tutorial will cover a few basic and advanced concepts when working with the core SBOL libraries. Primarily, we will learn how to read and write SBOL documents, create new
devices, attach experiemental data to a device, and interact with external biological parts repositories such as SynBioHub. The high level goal is to create a new device from components drawn from different sources, attach experiement data to that device, and upload that device to a remote repository.


## Prework
1. Create an account on SynBioHub
2. Download the data file they will need to attach
3. Download the SBOL document with the second part


## Python Installation
`pip install pysbol`


# Getting a Device from an SBOL Compliant XML
In this section, we will read in a new device from an SBOL compliant XML and explore its contents.


In [1]:

from sbol import *

# Set your Homespace. All new SBOL objects will be created in this namespace
my_namespace = 'http://my_namespace.org'  # Ex: http://my_namespace.org
setHomespace(my_namespace)

# Start a new SBOL Document to hold the device
doc = Document()
my_device = doc.componentDefinitions.create('my_device')
print(my_device)

# Load some genetic parts taken from the Cello paper
cello_parts = Document('cello_parts.xml')


# Inspect the Document's contents
print(len(cello_parts))
print(cello_parts)


http://my_namespace.org/ComponentDefinition/my_device/1
32
Attachment....................0
Collection....................4
CombinatorialDerivation.......0
ComponentDefinition...........14
Implementation................0
Model.........................0
ModuleDefinition..............0
Sequence......................14
Analysis......................0
Build.........................0
Design........................0
SampleRoster..................0
Test..........................0
Activity......................0
Agent.........................0
Plan..........................0
Annotation Objects............0
---
Total.........................32



In [2]:

# Read in the XML and explore its contents. Notice it is composed of
# componentDefinitions and sequences
for obj in cello_parts:
    print(obj)


http://examples.org/Sequence/pPhlF_sequence/1
http://examples.org/Sequence/pAmeR_sequence/1
http://examples.org/Sequence/YFP_protein_sequence/1
http://examples.org/Sequence/S4_SrpR_sequence/1
http://examples.org/Sequence/Q2_sequence/1
http://examples.org/Sequence/P3_PhlF_sequence/1
http://examples.org/Sequence/N1_sequence/1
http://examples.org/Sequence/LuxR_sequence/1
http://examples.org/Sequence/AmeR_sequence/1
http://examples.org/ComponentDefinition/pLuxStar/1
http://examples.org/ComponentDefinition/YFP/1
http://examples.org/Sequence/pLuxStar_sequence/1
http://examples.org/ComponentDefinition/pAmeR/1
http://examples.org/ComponentDefinition/LuxR/1
http://examples.org/ComponentDefinition/S4_SrpR/1
http://examples.org/ComponentDefinition/PhlF/1
http://examples.org/Sequence/ECK120010818_sequence/1
http://examples.org/ComponentDefinition/pPhlF/1
http://examples.org/Sequence/L3S2P55_sequence/1
http://examples.org/ComponentDefinition/L3S2P55/1
http://examples.org/ComponentDefinition/Q2/1
ht

StopIteration: 

In [3]:
# Import these objects into your Document
cello_parts.copy('http://examples.org', doc)
# Notice the objects have been imported into your Homespace
for obj in doc:
    print(obj)

 http://my_namespace.org/Sequence/PhlF_sequence/1
http://my_namespace.org/Collection/rbs/1
http://my_namespace.org/ComponentDefinition/ECK120010818/1
http://my_namespace.org/ComponentDefinition/P3_PhlF/1
http://my_namespace.org/ComponentDefinition/S4_SrpR/1
http://my_namespace.org/ComponentDefinition/Q2/1
http://my_namespace.org/Sequence/ECK120010818_sequence/1
http://my_namespace.org/ComponentDefinition/L3S3P11/1
http://my_namespace.org/ComponentDefinition/PhlF/1
http://my_namespace.org/Collection/terminators/1
http://my_namespace.org/ComponentDefinition/LuxR/1
http://my_namespace.org/Sequence/L3S3P11_sequence/1
http://my_namespace.org/ComponentDefinition/pAmeR/1
http://my_namespace.org/Collection/promoters/1
http://my_namespace.org/Collection/coding_sequences/1
http://my_namespace.org/Sequence/pLuxStar_sequence/1
http://my_namespace.org/ComponentDefinition/YFP/1
http://my_namespace.org/Sequence/pAmeR_sequence/1
http://my_namespace.org/Sequence/AmeR_sequence/1
http://my_namespace.org/

StopIteration: 

In [4]:
# Retrieve an object from the Document using its uniform resource identifier (URI)
promoter_collection = doc.getCollection(my_namespace + '/Collection/' + 'promoters')

# A Collection contains a list of URI references to objects, not the object themselves
for p in promoter_collection.members:
    print(p)

# Retrieve a component, using its full URI
promoter = doc.getComponentDefinition(my_namespace + '/ComponentDefinition/pPhlF/1')


# Retrieve the same component, using its displayId
promoter = doc.componentDefinitions['pPhlF']

# Review the BioPAX and Sequence Ontology terms that describe this component
print(promoter.types)
print(promoter.roles)

 http://my_namespace.org/ComponentDefinition/pAmeR/1
http://my_namespace.org/ComponentDefinition/pLuxStar/1
http://my_namespace.org/ComponentDefinition/pPhlF/1
['http://www.biopax.org/release/biopax-level3.owl#DnaRegion']
['http://identifiers.org/so/SO:0000167']


# Getting a Device from Synbiohub
In this section, we are going to download a device from SynBioHub. We want the medium strength promoter device from the iGEM interlab study. This device will contain a number of components, sequences, and other objects as well.

In [5]:
# Start an interface to the part shop
part_shop = sbol.PartShop('https://synbiohub.org/public/igem')


In [6]:
# Search for records from the interlab study
records = part_shop.search('interlab')
for record in records:
    print('{}: {}'.format(record.displayId, record))

Medium_2016Interlab: https://synbiohub.org/public/iGEM_2016_interlab/Medium_2016Interlab/1
Negative_2016Interlab: https://synbiohub.org/public/iGEM_2016_interlab/Negative_2016Interlab/1
Positive_2016Interlab: https://synbiohub.org/public/iGEM_2016_interlab/Positive_2016Interlab/1
Strong_2016Interlab: https://synbiohub.org/public/iGEM_2016_interlab/Strong_2016Interlab/1
Weak_2016Interlab: https://synbiohub.org/public/iGEM_2016_interlab/Weak_2016Interlab/1


In [7]:
# Import the medium device into the user's Document
medium_comp_uri = records[0].identity
part_shop.pull(medium_comp_uri, doc)

In [8]:


# Explore the new parts
for obj in doc:
    print('{}: {}'.format(sbol.parseClassName(obj.type), obj))

Sequence: http://my_namespace.org/Sequence/BBa_B0012_sequence/1
Agent: https://synbiohub.org/public/SBOL_Software/SBOLDesigner/3.0
Sequence: http://my_namespace.org/Sequence/BBa_B0034_sequence/1
ComponentDefinition: http://my_namespace.org/ComponentDefinition/BBa_B0010/1
ComponentDefinition: http://my_namespace.org/ComponentDefinition/BBa_B0034/1
ComponentDefinition: http://my_namespace.org/ComponentDefinition/pSB1C3/1
ComponentDefinition: http://my_namespace.org/ComponentDefinition/BBa_B0015/1
Sequence: https://synbiohub.org/public/iGEM_2016_interlab/Medium_2016InterlabSequence/1
ComponentDefinition: http://my_namespace.org/ComponentDefinition/BBa_J23106/1
ComponentDefinition: http://my_namespace.org/ComponentDefinition/BBa_B0012/1
Sequence: http://my_namespace.org/Sequence/BBa_E0040_sequence/1
Activity: http://my_namespace.org/Activity/igem2sbol/1
Sequence: http://my_namespace.org/Sequence/PhlF_sequence/1
Collection: http://my_namespace.org/Collection/rbs/1
Sequence: http://my_namesp


# Extracting a ComponentDefinition from a Pre-existing Device
In this section, we will extract the medium strength promoter from the interlab study and add it to the cassette document


In [9]:


# Extract the medium strength promoter
medium_strength_promoter = doc.componentDefinitions['BBa_J23106']

In [10]:
# Get parts for a new circuit
rbs = doc.componentDefinitions['LuxR']
cds = doc.componentDefinitions['Q2']
terminator = doc.componentDefinitions['ECK120010818']


# Creating a New Device
In this section, we will create a new device by swapping in the promoter from the device in interlab study into the device from the XML. We will start by grabbing the necessary parts from cassette document, and then assembling them together into a new device.


In [11]:
# Assemble a new gene
my_device.assemblePrimaryStructure([ medium_strength_promoter, rbs, cds, terminator ])

In [12]:
# Annotate the target construct with a Sequence Ontology term
my_device.roles = sbol.SO_GENE

In [13]:
# Explore the newly assembled gene
for comp in my_device.getPrimaryStructure():
    print(comp.displayId)

BBa_J23106
LuxR
Q2
ECK120010818


In [14]:
target_sequence = my_device.compile()
print(my_device.sequence.elements)

tttacggctagctcagtcctaggtatagtgctagcATGAACATTAAAAATATAAATGCTAATGAGAAGATAATTGATAAAATTAAAACTTGTAATAATAATAAAGATATTAATCAATGTTTATCTGAAATAGCAAAGATAATACATTGTGAATATTACCTATTCGCTATTATCTATCCTCACTCAATAATTAAACCTGATGTTTCAATTATAGATAATTACCCTGAAAAATGGCGTAAATATTATGATGATGCCGGACTACTAGAATATGACCCTGTAGTCGATTACTCTAAGTCCCATCATTCACCAATTAATTGGAACGTATTCGAAAAAAAAACAATAAAAAAAGAGTCTCCGAATGTAATAAAAGAAGCACAGGAATCGGGACTCATTACTGGATTTAGCTTTCCAATTCATACTGCAAGTAATGGTTTTGGAATGCTCAGTTTTGCTCATTCAGATAAAGATATTTATACTGACAGTTTATTTTTACACGCTAGTACAAATGTACCATTAATGCTTCCTTCTTTAGTCGATAATTATCAAAAAATAAATACGACACGTAAAAAGTCAGATTCTATTTTAACAAAAAGAGAAAAAGAATGCTTAGCGTGGGCGAGTGAAGGAAAAAGTACATGGGATATTTCAAAAATACTTGGCTGCAGTGAGCGTACTGTCACTTTTCATTTAACCAATACTCAAATGAAACTCAATACAACTAACCGCTGCCAAAGTATTTCTAAAGCAATTTTAACTGGCGCCATTAATTGTCCATACCTTAAAAATTAAGCCATGCCATTGGCTTTTGATAGAGGACAACTACTAGGTCAGTTTCACCTGTTTTACGTAAAAACCCGCTTCGGCGGGTTTTTACTTTTGG


# Managing a Design-Build-Test-Learn workflow
Now that we have a target construct, we will hand the design off to the laboratory 
for construction and characterization. To represent a workflow we use Activity objects.
Each Activity has an Agent who executes a Plan.

In [15]:
design = doc.designs.create('my_device')
design.structure = my_device
design.function = None  # This tutorial does not cover ModuleDefinitions and interactions

workflow_step_1 = Activity('build_1')
workflow_step_2 = Activity('build_2')
workflow_step_3 = Activity('test_1')
workflow_step_4 = Activity('analysis_1')

workflow_step_1.plan = Plan('gibson_assembly')
workflow_step_2.plan = Plan('transformation')
workflow_step_3.plan = Plan('promoter_characterization')
workflow_step_4.plan = Plan('parameter_optimization')

setHomespace('')
Config.setOption('sbol_compliant_uris', False)  # Temporarily disable auto-construction of URIs

workflow_step_1.agent = Agent('mailto:jdoe@%s' %my_namespace)
workflow_step_2.agent = workflow_step_1.agent
workflow_step_3.agent = Agent('http://sys-bio.org/plate_reader_1')
workflow_step_4.agent = Agent('http://tellurium.analogmachine.org')

setHomespace(my_namespace)
Config.setOption('sbol_compliant_uris', True)


doc.addActivity([workflow_step_1, workflow_step_2, workflow_step_3, workflow_step_4])

gibson_mix = workflow_step_1.generateBuild('gibson_mix', design)
clones = workflow_step_2.generateBuild(['clone1', 'clone2', 'clone3'], design, gibson_mix)
experiment1 = workflow_step_3.generateTest('experiment1', clones)
analysis1 = workflow_step_4.generateAnalysis('analysis1', experiment1)

# Validate the Document

print(doc.validate())

Valid. 


In [16]:
print(analysis1.identity)

http://my_namespace.org/Analysis/analysis1/1


# Uploading the Device back to SynBioHub
Finally, we can create a new collection on SynBioHub with the new device and its workflow history.

In [17]:
import getpass
email = 'zeke7781@yahoo.com'
user_name = 'zeke778'
password = getpass.getpass()

········


In [18]:
part_shop.login(user_name, password)

In [19]:
# Upon submission, the Document will be converted to a Collection with the following properties
# The new Collection will have a URI that conforms to the following pattern:
# https://synbiohub.org/user/<USERNAME>/<DOC.DISPLAYID>/<DOC.DISPLAYID>_collection
doc.displayId = 'my_device'
doc.name = 'my device'
doc.description = 'a description of the cassette'


In [20]:
part_shop.submit(doc)

'Successfully uploaded'

In [25]:
attachment_path = '/Users/palchicz/projects/sbol/gene_cassette.xml'

# Attach raw experimental data to the Test object here. Note the pattern
test_uri = 'https://synbiohub.org/user/' + 'zeke7781' + '/' + doc.displayId + '/Test_experiment1/1'
part_shop.attachFile(test_uri, attachment_path)



In [27]:
# Attach processed experimental data here
other_attachement_path = '/Users/palchicz/projects/sbol/attachement.txt'
part_shop.attachFile('https://synbiohub.org/user/' + 'zeke7781' + '/' + doc.displayId + '/Analysis/analysis1/1', other_attachement_path)


RuntimeError: Attempt to attach file failed with HTTP 404